### Import packages

In [33]:
import numpy as np
import pandas as pd
import requests as requests
from io import StringIO
import csv
import os
from datetime import datetime
import pytz
from time import time
import matplotlib.pyplot as plt

### Prepering raw dataset for analysis
#### All information about dataset you can find in [README.MD](https://github.com/MSI17819/Berlin_bike_theft_prediction/blob/main/README.md) file

#### Create empty dataframe

In [34]:
df = pd.DataFrame()

#### Data pulling function from url adress, where we can find data about Berlin bikes thief record from 01/01/2021-up to today.
#### The data are updated everyday or at least once a two/three day, so when we run function the data are expanding on new records.

In [35]:
def data_pulling_function():
    global df
    url = "https://www.internetwache-polizei-berlin.de/vdb/Fahrraddiebstahl.csv"
    response = requests.get(url)
    df_raw = pd.read_csv(StringIO(response.text))
    local_time = datetime.now(pytz.timezone('Europe/Warsaw'))
    df = df.append(df_raw)
    pd.set_option('display.max_columns', None)
    print('Data update - {}'.format(local_time.strftime("%Y-%m-%d %H:%M:%S")))
    return df

data_pulling_function()

Data update - 2022-12-02 21:37:15


C:\Users\dell\AppData\Local\Temp/ipykernel_3944/789137709.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(df_raw)


,ANGELEGT_AM,TATZEIT_ANFANG_DATUM,TATZEIT_ANFANG_STUNDE,TATZEIT_ENDE_DATUM,TATZEIT_ENDE_STUNDE,LOR,SCHADENSHOEHE,VERSUCH,ART_DES_FAHRRADS,DELIKT,ERFASSUNGSGRUND
0,01.12.2022,01.12.2022,8,01.12.2022,8,1300836,506,Nein,Fahrrad,Fahrraddiebstahl,Sonstiger schwerer Diebstahl von Fahrrädern
1,01.12.2022,29.11.2022,22,30.11.2022,17,1300732,718,Nein,diverse Fahrräder,Fahrraddiebstahl,Sonstiger schwerer Diebstahl von Fahrrädern
2,01.12.2022,28.05.2022,7,28.05.2022,14,6300526,569,Nein,Kinderfahrrad,Fahrraddiebstahl,Sonstiger schwerer Diebstahl von Fahrrädern
3,01.12.2022,21.11.2022,9,21.11.2022,16,1100309,600,Nein,Kinderfahrrad,Fahrraddiebstahl,Sonstiger schwerer Diebstahl von Fahrrädern
4,01.12.2022,30.11.2022,16,01.12.2022,11,3701659,400,Nein,Herrenfahrrad,Fahrraddiebstahl,Sonstiger schwerer Diebstahl von Fahrrädern
...,...,...,...,...,...,...,...,...,...,...,...
40782,02.01.2021,02.01.2021,13,02.01.2021,13,8401243,1700,Nein,Herrenfahrrad,Fahrraddiebstahl,Sonstiger schwerer Diebstahl von Fahrrädern
40783,01.01.2021,01.01.2021,5,01.01.2021,8,10400941,2527,Nein,diverse Fahrräder,Keller- und Bodeneinbruch,Sonstiger schwerer Diebstahl in/aus Keller/Bod...
40784,01.01.2021,01.01.2021,17,01.01.2021,18,8401241,888,Nein,Fahrrad,Fahrraddiebstahl,Einfacher Diebstahl von Fahrrädern
40785,01.01.2021,01.01.2021,9,01.01.2021,11,4200206,290,Nein,Damenfahrrad,Fahrraddiebstahl,Sonstiger schwerer Diebstahl von Fahrrädern


### Preprocesing raw data
- Change columns name 
-  Merging data category

#### Change columns names

In [36]:
df.columns = ["Record_date", "Stealing_date", "Stealing_hour", "Report_stealing_date",
              "Report_stealing_hour", "Berlin_code_area", "Bike_value", "Unsuccesful_attempt",
              "Bike_type", "Crime_type", "Crime_reason"]

#### Change data category in *Unsuccesful_attempt* for three categories

In [37]:
df["Unsuccesful_attempt"].replace({'Nein' : 'No', 'Ja' : 'Yes', 'Unbekannt' : 'Unknown'}, inplace=True)

#### Merge two category *Fahrrad* and *diverse Fahrrader* from *Bike_type* column as one category *Bike*

In [38]:
df["Bike_type"].replace({"Herrenfahrrad" : "Men", "Damenfahrrad" : "Women", "Fahrrad" : "Bike", 
                         "Mountainbike" : "Mountain", "Kinderfahrrad" : "Child", 
                         "diverse Fahrräder" : "Bike", "Rennrad" : "Racing", "Lastenfahrrad" : "Cargo"}, inplace=True)

#### Change column *Crime_type* vales

In [39]:
df["Crime_type"].replace(to_replace=["Fahrraddiebstahl", "Keller- und Bodeneinbruch"], 
                         value=["Theft", "Break-in"], inplace=True)

#### Change type of category *Crime_reason*
#### Category *aggrevated theft* = theft with break-in, theft with robbery (aggresor has to use force) 

In [40]:
df["Crime_reason"].replace(to_replace=["Sonstiger schwerer Diebstahl von Fahrrädern", 
                                       "Einfacher Diebstahl von Fahrrädern",
                                       "Sonstiger schwerer Diebstahl in/aus Keller/Boden von Fahrrädern", 
                                       "Einfacher Diebstahl aus Keller/Boden von Fahrrädern"],
                           value=["Aggravated theft", "Theft", "Aggravated theft from basement/floor", 
                                  "Theft from basement/floor"], inplace=True)

#### Changing column *Record_date* type from object to datetime format

In [41]:
df['Record_date'] = pd.to_datetime(df['Record_date'], dayfirst= True)

#### Create support column *date2* for next step spliting

In [42]:
df['date2'] = pd.to_datetime(df['Record_date'], dayfirst= True)

#### Create three new columns with: day, month and year

In [43]:
df['Record_day'] = df['date2'].dt.day
                                    
df['Record_month'] = df['date2'].dt.month

df['Record_year'] = df['date2'].dt.year

#### Moving three new columns for secound, third and fourth position in DataFrame

In [44]:
col_to_move = df.pop('Record_day')
col_to_move_1 = df.pop('Record_month')
col_to_move_2 = df.pop('Record_year')

df.insert(1, 'Record_day', col_to_move)
df.insert(2, 'Record_month', col_to_move_1)
df.insert(3, 'Record_year', col_to_move_2)

#### Changing *Stealing_date* type from object to datetime format

In [45]:
df['Stealing_date'] = pd.to_datetime(df['Stealing_date'], dayfirst= True)

#### Changing *Stealing_date* type from object to datetime format

In [46]:
df['Report_stealing_date'] = pd.to_datetime(df['Report_stealing_date'], dayfirst= True)

#### Change format into two columns (*Report_stealing_hour'*, *Stealing_hour*) from object to datetime hour format

In [47]:
df['Stealing_hour'] = pd.to_datetime(df['Stealing_hour'], format='%H').dt.hour

df['Report_stealing_hour'] = pd.to_datetime(df['Report_stealing_hour'],  format='%H').dt.hour 

#### Drop support colum *date2* from DataFrame

In [48]:
df.drop(['date2'], axis=1, inplace=True)

#### Display new DataFrame

In [49]:
df

,Record_date,Record_day,Record_month,Record_year,Stealing_date,Stealing_hour,Report_stealing_date,Report_stealing_hour,Berlin_code_area,Bike_value,Unsuccesful_attempt,Bike_type,Crime_type,Crime_reason
0,2022-12-01,1,12,2022,2022-12-01,8,2022-12-01,8,1300836,506,No,Bike,Theft,Aggravated theft
1,2022-12-01,1,12,2022,2022-11-29,22,2022-11-30,17,1300732,718,No,Bike,Theft,Aggravated theft
2,2022-12-01,1,12,2022,2022-05-28,7,2022-05-28,14,6300526,569,No,Child,Theft,Aggravated theft
3,2022-12-01,1,12,2022,2022-11-21,9,2022-11-21,16,1100309,600,No,Child,Theft,Aggravated theft
4,2022-12-01,1,12,2022,2022-11-30,16,2022-12-01,11,3701659,400,No,Men,Theft,Aggravated theft
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40782,2021-01-02,2,1,2021,2021-01-02,13,2021-01-02,13,8401243,1700,No,Men,Theft,Aggravated theft
40783,2021-01-01,1,1,2021,2021-01-01,5,2021-01-01,8,10400941,2527,No,Bike,Break-in,Aggravated theft from basement/floor
40784,2021-01-01,1,1,2021,2021-01-01,17,2021-01-01,18,8401241,888,No,Bike,Theft,Theft
40785,2021-01-01,1,1,2021,2021-01-01,9,2021-01-01,11,4200206,290,No,Women,Theft,Aggravated theft


#### Load csv file with LOR (Lebensweltlich orientierte Räume (LOR) = Spatial Planning Unit for Berlin) 
#### The dataset was prepared in advance. File contain columns: 
- *Berlin_code_area*
- *Administrative_district*
- *District* 
- *Quater*
- *Street_name* 

#### Load data about LOR (Lebensweltlich orientierte Räume (LOR) = Spatial Planning Unit

In [50]:
url = r"https://raw.githubusercontent.com/MSI17819/Berlin_bike_theft_prediction/main/Berlin_LOR_code.csv"

df_region = pd.read_csv(url, sep=';', encoding="ISO-8859-2")

df_region.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 542 entries, 0 to 541
Data columns (total 5 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   Berlin_code_area         542 non-null    int64 
 1   Administrative_district  542 non-null    object
 2   District                 542 non-null    object
 3   Quarter                  542 non-null    object
 4   Street_name              542 non-null    object
dtypes: int64(1), object(4)
memory usage: 21.3+ KB


#### Join two data frame (*left join*) with prepared data and LOR dataset

In [51]:
df_merge = pd.merge(df, df_region, how='left', on='Berlin_code_area')

#### Inspection DataFrame after merging two datasets 

In [52]:
df_merge

,Record_date,Record_day,Record_month,Record_year,Stealing_date,Stealing_hour,Report_stealing_date,Report_stealing_hour,Berlin_code_area,Bike_value,Unsuccesful_attempt,Bike_type,Crime_type,Crime_reason,Administrative_district,District,Quarter,Street_name
0,2022-12-01,1,12,2022,2022-12-01,8,2022-12-01,8,1300836,506,No,Bike,Theft,Aggravated theft,Mitte,Gesundbrunnen,Brunnenstraße Nord,Humboldthain Nordwest
1,2022-12-01,1,12,2022,2022-11-29,22,2022-11-30,17,1300732,718,No,Bike,Theft,Aggravated theft,Mitte,Gesundbrunnen,Osloer Straße,Soldiner Straße
2,2022-12-01,1,12,2022,2022-05-28,7,2022-05-28,14,6300526,569,No,Child,Theft,Aggravated theft,Steglitz-Zehlendorf,Zehlendorf Süd/Lichterfelde West,Teltower Damm,Zehlendorf Mitte
3,2022-12-01,1,12,2022,2022-11-21,9,2022-11-21,16,1100309,600,No,Child,Theft,Aggravated theft,Mitte,Zentrum,Alexanderplatz,Oranienburger Straße
4,2022-12-01,1,12,2022,2022-11-30,16,2022-12-01,11,3701659,400,No,Men,Theft,Aggravated theft,Pankow,Südlicher Prenzlauer Berg,Prenzlauer Berg Süd,Immanuelkirchstraße
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40782,2021-01-02,2,1,2021,2021-01-02,13,2021-01-02,13,8401243,1700,No,Men,Theft,Aggravated theft,Neukölln,Buckow Nord/Rudow,Rudow,Alt-Rudow
40783,2021-01-01,1,1,2021,2021-01-01,5,2021-01-01,8,10400941,2527,No,Bike,Break-in,Aggravated theft from basement/floor,Marzahn-Hellersdorf,Kaulsdorf/Mahlsdorf,Mahlsdorf,Pilgramer Straße
40784,2021-01-01,1,1,2021,2021-01-01,17,2021-01-01,18,8401241,888,No,Bike,Theft,Theft,Neukölln,Buckow Nord/Rudow,Rudow,Blumenviertel
40785,2021-01-01,1,1,2021,2021-01-01,9,2021-01-01,11,4200206,290,No,Women,Theft,Aggravated theft,Charlottenburg-Wilmersdorf,Charlottenburg West,Heerstraße,Kranzallee


### Cleaning data

#### Duplicates inspection

In [53]:
df_merge.duplicated().value_counts()

False    40714
True        73
dtype: int64

#### Deleting duplicates

In [54]:
 df_merge.drop_duplicates(keep='first', inplace=True)

#### DataFrame inspection after deleting duplicated rows

In [55]:
df_merge.duplicated().value_counts()

False    40714
dtype: int64

#### DataFrame inspection about missing values

In [56]:
df_merge.isna().sum()

Record_date                0
Record_day                 0
Record_month               0
Record_year                0
Stealing_date              0
Stealing_hour              0
Report_stealing_date       0
Report_stealing_hour       0
Berlin_code_area           0
Bike_value                 0
Unsuccesful_attempt        0
Bike_type                  0
Crime_type                 0
Crime_reason               0
Administrative_district    0
District                   0
Quarter                    0
Street_name                0
dtype: int64

#### Inspection about category *Unsuccesfull_attempt*

In [57]:
df_merge['Unsuccesful_attempt'].value_counts()

No         40556
Yes          138
Unknown       20
Name: Unsuccesful_attempt, dtype: int64

#### Filter data only for those where thiefs happen and unhappen (*Unsuccesfull_attempt == No, Unsuccesfull_attempt == Yes*), exclude data with *Unknow* category.

In [58]:
df_merge = df_merge[(df_merge['Unsuccesful_attempt'] == 'No') | (df_merge['Unsuccesful_attempt'] == 'Yes')]

#### Inspection DataFrame after droping record with *Unknown* category

In [59]:
df_merge['Unsuccesful_attempt'].value_counts()

No     40556
Yes      138
Name: Unsuccesful_attempt, dtype: int64

#### Reset index and save changes

In [60]:
df_merge.reset_index(drop=True, inplace=True)

#### Dispay DataFrame

In [64]:
df_merge

,Record_date,Record_day,Record_month,Record_year,Stealing_date,Stealing_hour,Report_stealing_date,Report_stealing_hour,Berlin_code_area,Bike_value,Unsuccesful_attempt,Bike_type,Crime_type,Crime_reason,Administrative_district,District,Quarter,Street_name
0,2022-12-01,1,12,2022,2022-12-01,8,2022-12-01,8,1300836,506,No,Bike,Theft,Aggravated theft,Mitte,Gesundbrunnen,Brunnenstraße Nord,Humboldthain Nordwest
1,2022-12-01,1,12,2022,2022-11-29,22,2022-11-30,17,1300732,718,No,Bike,Theft,Aggravated theft,Mitte,Gesundbrunnen,Osloer Straße,Soldiner Straße
2,2022-12-01,1,12,2022,2022-05-28,7,2022-05-28,14,6300526,569,No,Child,Theft,Aggravated theft,Steglitz-Zehlendorf,Zehlendorf Süd/Lichterfelde West,Teltower Damm,Zehlendorf Mitte
3,2022-12-01,1,12,2022,2022-11-21,9,2022-11-21,16,1100309,600,No,Child,Theft,Aggravated theft,Mitte,Zentrum,Alexanderplatz,Oranienburger Straße
4,2022-12-01,1,12,2022,2022-11-30,16,2022-12-01,11,3701659,400,No,Men,Theft,Aggravated theft,Pankow,Südlicher Prenzlauer Berg,Prenzlauer Berg Süd,Immanuelkirchstraße
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40689,2021-01-02,2,1,2021,2021-01-02,13,2021-01-02,13,8401243,1700,No,Men,Theft,Aggravated theft,Neukölln,Buckow Nord/Rudow,Rudow,Alt-Rudow
40690,2021-01-01,1,1,2021,2021-01-01,5,2021-01-01,8,10400941,2527,No,Bike,Break-in,Aggravated theft from basement/floor,Marzahn-Hellersdorf,Kaulsdorf/Mahlsdorf,Mahlsdorf,Pilgramer Straße
40691,2021-01-01,1,1,2021,2021-01-01,17,2021-01-01,18,8401241,888,No,Bike,Theft,Theft,Neukölln,Buckow Nord/Rudow,Rudow,Blumenviertel
40692,2021-01-01,1,1,2021,2021-01-01,9,2021-01-01,11,4200206,290,No,Women,Theft,Aggravated theft,Charlottenburg-Wilmersdorf,Charlottenburg West,Heerstraße,Kranzallee


In [30]:
df_merge.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40694 entries, 0 to 40693
Data columns (total 18 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   Record_date              40694 non-null  datetime64[ns]
 1   Record_day               40694 non-null  int64         
 2   Record_month             40694 non-null  int64         
 3   Record_year              40694 non-null  int64         
 4   Stealing_date            40694 non-null  datetime64[ns]
 5   Stealing_hour            40694 non-null  int64         
 6   Report_stealing_date     40694 non-null  datetime64[ns]
 7   Report_stealing_hour     40694 non-null  int64         
 8   Berlin_code_area         40694 non-null  int64         
 9   Bike_value               40694 non-null  int64         
 10  Unsuccesful_attempt      40694 non-null  object        
 11  Bike_type                40694 non-null  object        
 12  Crime_type               40694 n

#### Export DataFrame to csv file for future prediction analysis

In [32]:
df_merge.to_csv(r'C:\Users\dell\Desktop\Project\Berlin_bike_theft_prediction\Berlin_bike_thief_file_for_analysis.csv', 
                sep=',', encoding='utf-8', header=True, index=False)